In [ ]:
import chainconsumer
import gc
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os

import sys
sys.path.append('/dipc/kstoreyf/muchisimocks/scripts')
#import sbi_tools
import plot_utils
#import scripts
# from scripts import sbi_tools
#from scripts import plot_utils

from momentnetworks import demo

%load_ext autoreload
%autoreload 2

%matplotlib inline
mpl.pyplot.style.use('default')
mpl.pyplot.close('all')

font, rcnew = plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
mpl.pyplot.rcParams.update(rcnew)
mpl.pyplot.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

#N_threads = sbi_tools.set_N_threads(6)

### Load baccoemu P(k) data

In [ ]:
tag_emuPk = '_2param'

fn_emuPk = f'../data/emuPks/emuPks{tag_emuPk}.npy'
fn_emuPk_params = f'../data/emuPks/emuPks_params{tag_emuPk}.txt'
fn_emuk = f'../data/emuPks/emuPks_k{tag_emuPk}.txt'

Pk = np.load(fn_emuPk)
theta = np.genfromtxt(fn_emuPk_params, delimiter=',', names=True)
param_names = theta.dtype.names
# from tuples to 2d array
theta = np.array([list(tup) for tup in theta])
kk = np.genfromtxt(fn_emuk)

In [ ]:
mask = np.full(len(kk), True)

In [ ]:
print(Pk.shape, theta.shape)

In [ ]:
# if wan't to run on a subset, edit n_samples here (max=1000 right now)
n_samples = 1000 
Pk = Pk[:n_samples]
theta = theta[:n_samples]

In [ ]:
n_tot = theta.shape[0]
n_params = theta.shape[1]
n_dim = Pk.shape[1]
print(n_tot, n_params, n_dim)

Plot P(k) data:

In [ ]:
fig, ax = mpl.pyplot.subplots(figsize=(8, 6))
for iLH in range(n_tot):
    ax.loglog(kk[mask], Pk[iLH][mask])

ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
#n_biasmodels = len(biases_vec)
n_biasmodels = 0
n_cosmos = n_params
print(n_biasmodels, n_cosmos)

Split into train-val-test

In [ ]:
p_train, p_test = 0.8, 0.1
p_val = 1-p_train-p_test
train_split = int(theta.shape[0]*p_train)
test_split = int(theta.shape[0]*(1-p_test))
#train_val_split = int(n_biasmodels*round(theta.shape[0]*0.99/n_biasmodels))

theta_train = theta[:train_split]
theta_val = theta[train_split:test_split]
theta_test = theta[test_split:]
print(theta_train.shape, theta_val.shape, theta_test.shape)

Pk_train = Pk[:train_split]
Pk_val = Pk[train_split:test_split]
Pk_test = Pk[test_split:]

mask = np.all(Pk_train>0, axis=0)
Pk_train = Pk_train[:,mask]
Pk_val = Pk_val[:,mask]
Pk_test = Pk_test[:,mask]
k = kk[mask]

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1, figsize=(7,5))
fontsize = 24
fontsize1 = 18

alpha = 1

tmp_Pk_plot = Pk_train
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='royalblue', alpha=alpha, lw=0.5, label='training set')

tmp_Pk_plot = Pk_test
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='k', alpha=alpha, lw=0.5, label='test set')
    
ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
dict_bounds = {}
for pp, param_name in enumerate(param_names):
    dict_bounds[param_name] = [np.min(theta[:,pp]), np.max(theta[:,pp])]

In [ ]:
class Scaler:

    def __init__(self):
          pass
        
    def fit(self, x_train):
        self.x_train_min = np.min(x_train)
        self.x_train_max = np.max(x_train)
           
    def scale(self, x):
        log_x = np.log10(x)
        log_x_norm = (log_x - np.log10(self.x_train_min)) / (np.log10(self.x_train_max) - np.log10(self.x_train_min))
        return log_x_norm
    
    def unscale(self, x_scaled):
        x = x_scaled * (np.log10(self.x_train_max) - np.log10(self.x_train_min)) + np.log10(self.x_train_min)
        return 10**x  

In [ ]:
scaler = Scaler()
scaler.fit(Pk_train)
Pk_train_scaled = scaler.scale(Pk_train)
Pk_val_scaled = scaler.scale(Pk_val)
Pk_test_scaled = scaler.scale(Pk_test)

In [ ]:
print(np.min(Pk_train), np.max(Pk_train))
print(np.min(Pk_train_scaled), np.max(Pk_train_scaled))

print(np.min(Pk_test), np.max(Pk_test))
print(np.min(Pk_test_scaled), np.max(Pk_test_scaled))

In [ ]:
print(Pk_train.shape)
print(theta_train.shape)
print(n_params)

### Set up and run Moment Network model

Following demos at https://github.com/NiallJeffrey/MomentNetworks/tree/master

In [ ]:
model_instance = demo.simple_leaky(n_dim, n_params, learning_rate=1e-4) 
regression = model_instance.model() 

In [ ]:
print(theta_train.shape, Pk_train.shape)
print(theta_val.shape, Pk_val.shape)

Train initial model (basic MLP), as usual, on labeled data

In [ ]:
history = regression.fit(Pk_train_scaled, theta_train,
                         epochs=200, batch_size=32, shuffle=True,
                         validation_data=(Pk_val_scaled, theta_val))

In [ ]:
#predicted_mean = regression.predict(np.atleast_2d(Pk_train_scaled)) # maybe should be train & val??

Get means and residuals

In [ ]:
training_var_unknown_mean = (theta_train-regression.predict(np.atleast_2d(Pk_train_scaled)))**2.
training_var_unknown_mean = np.hstack([training_var_unknown_mean,
                                      np.atleast_2d((theta_train[:,0]-regression.predict(np.atleast_2d(Pk_train_scaled))[:,0])*
                                      (theta_train[:,1]-regression.predict(np.atleast_2d(Pk_train_scaled))[:,1])).T])
print(training_var_unknown_mean.shape)

training_var_unknown_mean_val = (theta_val-regression.predict(np.atleast_2d(Pk_val_scaled)))**2.
training_var_unknown_mean_val = np.hstack([training_var_unknown_mean_val,
                                      np.atleast_2d((theta_val[:,0]-regression.predict(np.atleast_2d(Pk_val_scaled))[:,0])*
                                      (theta_val[:,1]-regression.predict(np.atleast_2d(Pk_val_scaled))[:,1])).T])

Set up and train model on the residuals

In [ ]:
model_instance = demo.simple_leaky(n_dim, 3, learning_rate=1e-3)
regression_var_unknown_mean = model_instance.model()

In [ ]:
history = regression_var_unknown_mean.fit(Pk_train_scaled,
                                          training_var_unknown_mean,
                                          epochs=200, batch_size=32, shuffle=True,
                                          validation_data = (Pk_val_scaled,
                                                             training_var_unknown_mean_val))

### Test on a model pulled directly from the training set (NOT held-out data) 

In [ ]:

#idx_train_check = rng.choice(np.arange(len(theta_train)))
idx_train_check = 17

print(idx_train_check)
theta_train_check = np.array([theta_train[idx_train_check]])
print(theta_train_check)
Pk_train_scaled_check = np.array([Pk_train_scaled[idx_train_check]])

predicted_mean_obs = regression.predict(np.atleast_2d(Pk_train_scaled_check))
predicted_var_obs = (regression_var_unknown_mean.predict(np.atleast_2d(Pk_train_scaled_check))[0])

print(predicted_var_obs)
print(predicted_var_obs.shape)
moment_network_param_cov = np.empty((n_params, n_params))
moment_network_param_cov[0,0] = predicted_var_obs[0]
moment_network_param_cov[1,1] = predicted_var_obs[1]
moment_network_param_cov[0,1] = predicted_var_obs[2]
moment_network_param_cov[1,0] = predicted_var_obs[2]

In [ ]:
moment_network_samples = np.array(np.random.multivariate_normal(predicted_mean_obs[0],moment_network_param_cov,int(1e6)),dtype=np.float32)
gc.collect()

In [ ]:
param_label_dict = {'omega_cold': r'$\Omega_\mathrm{m}$',
                'sigma8_cold': r'$\sigma_{8}$',
                'hubble': r'$h$',
                'ns': r'$n_\mathrm{s}$',
                'omega_baryon': r'$\Omega_\mathrm{b}$',}
param_labels = [param_label_dict[param_name] for param_name in param_names]
extents = [dict_bounds[param_name] for param_name in param_names]

In [ ]:
c = chainconsumer.ChainConsumer()

c.add_chain(moment_network_samples,
            parameters=param_labels,
            name = '2D Moment Network result', color_params='C1',
            )

c.configure(kde=[1.,None],sigmas = [1,2],
            contour_label_font_size = 11,
            label_font_size = 16, shade = False) 

fig = c.plotter.plot(figsize = (5,4), 
                     extents=extents,
                     #extents=[[0, 0.8], [-0.6, 1.8]],
                     truth=theta_train_check[0]
                     )

### Test on a model from the test set (held-out data)

In [ ]:
idx_test = 0
predicted_mean_obs_test = regression.predict(np.atleast_2d(Pk_test_scaled[idx_test]))
predicted_var_obs_test = (regression_var_unknown_mean.predict(np.atleast_2d(Pk_test_scaled[idx_test]))[0])

moment_network_param_cov_test = np.empty((n_params, n_params))
moment_network_param_cov_test[0,0] = predicted_var_obs_test[0]
moment_network_param_cov_test[1,1] = predicted_var_obs_test[1]
moment_network_param_cov_test[0,1] = predicted_var_obs_test[2]
moment_network_param_cov_test[1,0] = predicted_var_obs_test[2]

In [ ]:
moment_network_samples_test = np.array(np.random.multivariate_normal(predicted_mean_obs_test[0],
                                  moment_network_param_cov_test,int(1e6)),dtype=np.float32)
gc.collect()

In [ ]:
c = chainconsumer.ChainConsumer()

c.add_chain(moment_network_samples_test,
            parameters=param_labels,
            name = '2D Moment Network result', color_params='C1',
            )

c.configure(kde=[1.,None],sigmas = [1,2],
            contour_label_font_size = 11,
            label_font_size = 16, shade = False) 

fig = c.plotter.plot(figsize = (5,4), 
                     extents=extents,
                     #extents=[[0, 0.8], [-0.6, 1.8]],
                     truth=theta_test[idx_test]
                     )